多输入通道

In [29]:
import torch 
from d2l import torch as d2l

In [30]:
def corr2d_multi_in(X, K):
    # 先遍历X和K的第0个维度(通道维度), 再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [31]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]],
                  [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

多输出通道

In [32]:
def corr2d_multi_in_out(X, K):
    # 迭代K的第0个维度，每次都对输入X执行互相关运算
    # 最后将所有结果都叠加在一起

    # stack，将多个具有相同形状的张量，在指定维度0上进行连接
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0) 

In [33]:
K = torch.stack((K, K+1, K+2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [34]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

1*1卷积层

In [35]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape  # 输入通道数，输入高，输入宽
    c_o = K.shape[0]     # 输出通道数
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [36]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))  # 2个输出通道，3个输入通道， 1*1卷积核

In [37]:
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1-Y2).sum()) < 1e-6